In [1]:
print('Installing torchquantum...')
!git clone https://github.com/mit-han-lab/torchquantum.git
%cd /content/torchquantum
!pip install --editable . 1>/dev/null

Installing torchquantum...
Cloning into 'torchquantum'...
remote: Enumerating objects: 15128, done.
remote: Counting objects: 100% (1837/1837), done.
remote: Compressing objects: 100% (466/466), done.
remote: Total 15128 (delta 1519), reused 1483 (delta 1369), pack-reused 13291
Receiving objects: 100% (15128/15128), 98.00 MiB | 22.87 MiB/s, done.
Resolving deltas: 100% (8593/8593), done.
/content/torchquantum


In [6]:
import torch
import torch.optim as optim
import torchquantum as tq
import torchquantum.functional as tqf
import numpy as np

class QuantumPulseDemo(tq.QuantumModule):
    def __init__(self):
        super().__init__()
        self.n_wires = 2

        # Quantum encoder
        self.encoder = tq.GeneralEncoder([
            {'input_idx': [0], 'func': 'rx', 'wires': [0]},
            {'input_idx': [1], 'func': 'rx', 'wires': [1]}
        ])

        # Define parameterized quantum pulse
        self.pulse = tq.pulse.QuantumPulseDirect(n_steps=4, hamil=[[0, 1], [1, 0]])

    def forward(self, x):
        qdev = tq.QuantumDevice(n_wires=self.n_wires, bsz=x.shape[0], device=x.device)
        self.encoder(qdev, x)
        self.apply_pulse(qdev)
        return tq.measure(qdev)

    def apply_pulse(self, qdev):
        pulse_params = self.pulse.pulse_shape.detach().cpu().numpy()
        # Apply pulse to the quantum device (adjust based on actual pulse application logic)
        for i, params in enumerate(pulse_params):
            tqf.rx(qdev, wires=0, params=params)
            tqf.rx(qdev, wires=1, params=params)

def main():
    # Set up input data and target unitary
    theta = 0.6
    target_unitary = torch.tensor(
        [
            [np.cos(theta / 2), -1j * np.sin(theta / 2)],
            [-1j * np.sin(theta / 2), np.cos(theta / 2)],
        ],
        dtype=torch.complex64,
    )

    # Initialize the model and optimizer
    model = QuantumPulseDemo()
    optimizer = optim.Adam(params=model.pulse.parameters(), lr=5e-3)

    # Training loop
    epochs = 1000
    for epoch in range(epochs):
        # Define input tensor (batch size 1, 2 features for 2 qubits)
        x = torch.tensor([[np.pi, np.pi]], dtype=torch.float32)

        # Forward pass
        qdev = model(x)

        # Compute loss
        loss = (
            1
            - (
                torch.trace(model.pulse.get_unitary() @ target_unitary)
                / target_unitary.shape[0]
            ).abs()
            ** 2
        )

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print progress
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')
            print('Current Pulse Shape:', model.pulse.pulse_shape)
            print('Current Unitary:', model.pulse.get_unitary())

if __name__ == "__main__":
    main()


Epoch 0, Loss: 0.8393601179122925
Current Pulse Shape: Parameter containing:
tensor([0.9950, 0.9950, 0.9950, 0.9950], requires_grad=True)
Current Unitary: tensor([[-0.6686+0.0000j,  0.0000+0.7436j],
        [ 0.0000+0.7436j, -0.6686+0.0000j]], grad_fn=<MmBackward0>)
Epoch 100, Loss: 0.00020205974578857422
Current Pulse Shape: Parameter containing:
tensor([0.7071, 0.7071, 0.7071, 0.7071], requires_grad=True)
Current Unitary: tensor([[-0.9514+0.0000j,  0.0000-0.3080j],
        [ 0.0000-0.3080j, -0.9514+0.0000j]], grad_fn=<MmBackward0>)
Epoch 200, Loss: 0.0
Current Pulse Shape: Parameter containing:
tensor([0.7104, 0.7104, 0.7104, 0.7104], requires_grad=True)
Current Unitary: tensor([[-0.9553+0.0000j,  0.0000-0.2955j],
        [ 0.0000-0.2955j, -0.9553+0.0000j]], grad_fn=<MmBackward0>)
Epoch 300, Loss: -2.384185791015625e-07
Current Pulse Shape: Parameter containing:
tensor([0.7104, 0.7104, 0.7104, 0.7104], requires_grad=True)
Current Unitary: tensor([[-0.9553+0.0000j,  0.0000-0.2955j],
 

In [7]:
import torch
import torch.optim as optim
import torchquantum as tq
import torchquantum.functional as tqf
import numpy as np

class QuantumPulseDemo(tq.QuantumModule):
    def __init__(self):
        super().__init__()
        self.n_wires = 2

        # Quantum encoder
        self.encoder = tq.GeneralEncoder([
            {'input_idx': [0], 'func': 'rx', 'wires': [0]},
            {'input_idx': [1], 'func': 'rx', 'wires': [1]}
        ])

        # Define parameterized quantum pulse
        self.pulse = tq.pulse.QuantumPulseDirect(n_steps=4, hamil=[[0, 1], [1, 0]])

    def forward(self, x):
        qdev = tq.QuantumDevice(n_wires=self.n_wires, bsz=x.shape[0], device=x.device)
        self.encoder(qdev, x)
        self.apply_pulse(qdev)
        return tq.measure(qdev)

    def apply_pulse(self, qdev):
        pulse_params = self.pulse.pulse_shape.detach().cpu().numpy()
        # Apply pulse to the quantum device (adjust based on actual pulse application logic)
        for params in pulse_params:
            tqf.rx(qdev, wires=0, params=params)
            tqf.rx(qdev, wires=1, params=params)

def main():
    # Set up input data and target unitary
    theta = 0.6
    target_unitary = torch.tensor(
        [
            [np.cos(theta / 2), -1j * np.sin(theta / 2)],
            [-1j * np.sin(theta / 2), np.cos(theta / 2)],
        ],
        dtype=torch.complex64,
    )

    # Initialize the model and optimizer
    model = QuantumPulseDemo()
    optimizer = optim.Adam(params=model.pulse.parameters(), lr=5e-3)

    # Training loop
    epochs = 1000
    for epoch in range(epochs):
        # Define input tensor (batch size 1, 2 features for 2 qubits)
        x = torch.tensor([[np.pi, np.pi]], dtype=torch.float32)

        # Forward pass
        qdev = model(x)

        # Compute loss
        loss = (
            1
            - (
                torch.trace(model.pulse.get_unitary() @ target_unitary)
                / target_unitary.shape[0]
            ).abs()
            ** 2
        )

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print progress
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')
            print('Current Pulse Shape:', model.pulse.pulse_shape)
            print('Current Unitary:\n', model.pulse.get_unitary())

if __name__ == "__main__":
    main()


Epoch 0, Loss: 0.8393601179122925
Current Pulse Shape: Parameter containing:
tensor([0.9950, 0.9950, 0.9950, 0.9950], requires_grad=True)
Current Unitary:
 tensor([[-0.6686+0.0000j,  0.0000+0.7436j],
        [ 0.0000+0.7436j, -0.6686+0.0000j]], grad_fn=<MmBackward0>)
Epoch 100, Loss: 0.00020205974578857422
Current Pulse Shape: Parameter containing:
tensor([0.7071, 0.7071, 0.7071, 0.7071], requires_grad=True)
Current Unitary:
 tensor([[-0.9514+0.0000j,  0.0000-0.3080j],
        [ 0.0000-0.3080j, -0.9514+0.0000j]], grad_fn=<MmBackward0>)
Epoch 200, Loss: 0.0
Current Pulse Shape: Parameter containing:
tensor([0.7104, 0.7104, 0.7104, 0.7104], requires_grad=True)
Current Unitary:
 tensor([[-0.9553+0.0000j,  0.0000-0.2955j],
        [ 0.0000-0.2955j, -0.9553+0.0000j]], grad_fn=<MmBackward0>)
Epoch 300, Loss: -2.384185791015625e-07
Current Pulse Shape: Parameter containing:
tensor([0.7104, 0.7104, 0.7104, 0.7104], requires_grad=True)
Current Unitary:
 tensor([[-0.9553+0.0000j,  0.0000-0.2955j

In [10]:
def apply_pulse(self, qdev):
    pulse_params = self.pulse.pulse_shape.detach().cpu().numpy()
    for params in pulse_params:
        tq.phase_shift(qdev, wires=0, params=np.pi)  # Apply pi phase shift before each pulse
        tqf.rx(qdev, wires=0, params=params)


In [15]:


class OM_EOM_Simulation:
    def __init__(self, pulse_duration, modulation_bandwidth=None, eom_mode=False):
        self.pulse_duration = pulse_duration
        self.modulation_bandwidth = modulation_bandwidth
        self.eom_mode = eom_mode

    def simulate_sequence(self):
        # Initialize the sequence
        sequence = []

        # Add pulses and delays to the sequence
        if self.modulation_bandwidth:
            # Apply modulation bandwidth effect
            sequence.append(('Delay', 0))
            sequence.append(('Pulse', 'NoisyChannel'))
        for _ in range(3):
            # Apply pulses with specified duration
            sequence.append(('Delay', self.pulse_duration))
            if self.eom_mode:
                # Apply EOM mode operation
                sequence.append(('Pulse', 'EOM'))
            else:
                # Apply regular pulse
                sequence.append(('Pulse', 'Regular'))
            # Apply a delay between pulses
            sequence.append(('Delay', 0))

        return sequence

# Define parameters for the simulation
pulse_duration = 100
modulation_bandwidth = 5
eom_mode = True

# Create the simulation object
sim = OM_EOM_Simulation(pulse_duration, modulation_bandwidth, eom_mode)

# Simulate the sequence
sequence = sim.simulate_sequence()

# Print the sequence
for step in sequence:
    print(step)


('Delay', 0)
('Pulse', 'NoisyChannel')
('Delay', 100)
('Pulse', 'EOM')
('Delay', 0)
('Delay', 100)
('Pulse', 'EOM')
('Delay', 0)
('Delay', 100)
('Pulse', 'EOM')
('Delay', 0)
